# 모델 앙상블

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Reshape

import time

# 회귀 문제

In [2]:
x = np.arange(-1, 1, 0.01)
np.random.shuffle(x)
y = np.sin(x)

i = int(x.shape[0]*0.8)

train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]


In [3]:
def build_model():  
  model = keras.Sequential()
  model.add(Input((1,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer="Adam", loss="mse")

  return model

In [4]:
n_members = 3

In [5]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [6]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

5/5 [==============================] - 0s 3ms/step - loss: 0.3558


In [7]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)
print("y_.shape =", y_.shape)
outcomes = np.mean(y_,axis=0).squeeze()
print("outcomes.shape =", outcomes.shape)

mse = tf.keras.losses.mean_squared_error(test_y, outcomes).numpy()
print("mse =", mse)

y_.shape = (3, 40, 1)
outcomes.shape = (40,)
mse = 0.22319138


# 분류 문제

In [8]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.mnist.load_data()

train_x = raw_train_x / 255.
test_x = raw_test_x / 255.

train_y = raw_train_y
test_y = raw_test_y

11501568/11490434 [==============================] - 0s 0us/step


In [9]:
def build_model():  
  model = keras.Sequential()
  model.add(Input((28,28)))
  model.add(Reshape((28,28,1)))
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  return model

In [10]:
n_members = 3

In [11]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [12]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32)

1875/1875 [==============================] - 6s 3ms/step - loss: 0.2920 - accuracy: 0.9080


In [15]:
from sklearn.metrics import accuracy_score

y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_)
summed = np.sum(y_, axis=0)
print(summed[0])
outcomes = np.argmax(summed, axis=1)
print(outcomes)

acc = accuracy_score(test_y, outcomes)
print(acc)


[1.5493895e-05 7.3752861e-05 1.2724465e-04 5.1777955e-04 4.6056293e-06
 3.2934577e-06 1.0122410e-04 2.9978621e+00 1.0133460e-04 1.1931964e-03]
[7 2 1 ... 4 5 6]
0.9773


# 2진 분류 문제

In [17]:
!wget https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/data/sonar.csv

--2021-12-28 07:11:42--  https://raw.githubusercontent.com/dhrim/MDC_2021/master/material/deep_learning/data/sonar.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87776 (86K) [text/plain]
Saving to: ‘sonar.csv’

sonar.csv           100%[===================>]  85.72K  --.-KB/s    in 0.02s   

2021-12-28 07:11:42 (5.02 MB/s) - ‘sonar.csv’ saved [87776/87776]



In [18]:
raw_data = pd.read_csv("sonar.csv")
data = raw_data.to_numpy()
np.random.shuffle(data)

x = data[:,:60]
y = data[:,60]

i = int(len(x)*0.8)
train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]

In [19]:
def build_model():  

  model = keras.Sequential()
  model.add(Dense(10, activation='relu', input_shape=(60,)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["acc"])

  return model

In [20]:
n_members = 3

In [21]:
models = []
for _ in range(n_members):
	model = build_model()
	models.append(model)

In [24]:
for i in range(n_members):
	models[i].fit(train_x, train_y, epochs=1, batch_size=32, validation_split=0.2)

5/5 [==============================] - 0s 111ms/step - loss: 0.6700 - acc: 0.6591 - val_loss: 0.6627 - val_acc: 0.6667


In [23]:
y_ = [model.predict(test_x) for model in models]
y_ = np.array(y_).squeeze()
predicted = (y_>0.5).astype(np.int)

import scipy
outcomes, _ = scipy.stats.mode(predicted)
print(outcomes[:10])
outcomes = outcomes.squeeze()


from sklearn.metrics import accuracy_score

acc = accuracy_score(test_y, outcomes)

[[1 0 0 1 0 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1
  0 1 1 0 1 0]]
